In [22]:
import re
from tqdm.auto import tqdm
tqdm.pandas()
import math
import pandas as pd

CORPUS_PATH = "../../data/MEDICAL/corpus"
PROCESSED_PATH = "../../processed"

In [23]:
# corpus clean
import re
import string

def remove_url(text):
  return re.sub(r"http\S+", "", text)

def remove_html_tags(text):
  clean = re.compile('<.*?>')
  return re.sub(clean, '', text)

codes = ["&#8211", "&#038"]
def remove_special_token(text):
  for code in codes:
    text = text.replace(code, " ")
  return text

def remove_punctation(text):
  for punctuation in string.punctuation:
    text = text.replace(punctuation, ' ')
  return text

def preprocess_text(text):
  text = " ".join(remove_punctation(remove_special_token(remove_html_tags(remove_url(text)))).split())
  return text

def clean_overall(doc):
  # remove corpus header
  try:
    doc = doc.split("<h3>Mục lục</h3>")[1]
  except:
    pass
  if "hệ thống bệnh viện đa khoa tâm anh" in doc.lower():
    process_paragraph = doc.lower().split("hệ thống bệnh viện đa khoa tâm anh")[0]
    doc = doc[ : len(process_paragraph)]
  return preprocess_text(doc)


In [24]:
# corpus breaks into chunks
_WORD_SPLIT = re.compile("([.,!?\"/':;)(])")
STOP_WORDS = "\" \' [ ] . , ! : ; ?".split(" ")

def basic_tokenizer(sentence):
    """Very basic tokenizer: split the sentence into a list of tokens."""
    words = []
    for space_separated_fragment in sentence.strip().split():
        words.extend(_WORD_SPLIT.split(space_separated_fragment))
        # return [w.lower() for w in words if w not in stop_words and w != '' and w != ' ']
    return [w.lower() for w in words if w != '' and w != ' ' and w not in string.punctuation]

def create_sliding_window(text, size=256, overlap=32):
    actual_size = size - overlap
    windows = []
    n_windows = math.ceil(len(text)/actual_size)
    for i in range(n_windows):
        windows.append(" ".join(text[i*actual_size:i*actual_size + size]))
    return windows

In [25]:
def get_titles(corpus):
    titles = []
    for doc in tqdm(corpus):
        title = doc.split("\n")[5].split(":")[0].split("?")[0]
        for code in codes:
            title = title.replace(code, " ")
        title = preprocess_text(title)
        titles.append(title)
    return titles

In [33]:
import os

all_titles = []
all_texts = []
# all_bm25_texts = []

print("read corpus")
corpus = []
for file_name in os.listdir(CORPUS_PATH):
    with open(f'{CORPUS_PATH}/{file_name}', 'r') as f:
        doc = f.readlines()
    corpus.append(" ".join(doc))

print("read titles")
titles = get_titles(corpus)

print("process")
for i, (title, text) in tqdm(enumerate(zip(titles, corpus))):
    # print(y, "|", text[:50],"\n")
    text = clean_overall(text)
    text = text.split(" ")
    sliding_windows = create_sliding_window(text)
    # bm25_windows = [" ".join(basic_tokenizer(w)) for w in sliding_windows]
    all_texts.extend(sliding_windows)
    # all_bm25_texts.extend(bm25_windows)
    all_titles.extend([title,]*len(sliding_windows))

read corpus
read titles


100%|██████████| 603/603 [00:00<00:00, 54044.30it/s]


process


603it [00:00, 1982.58it/s]


In [34]:
df = pd.DataFrame()
df["title"] = all_titles
df["text"] = all_texts

In [35]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=12)
df["bm25_text"] = df["text"].parallel_apply(lambda x: " ".join(basic_tokenizer(x)))

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [36]:
df.head()

,title,text,bm25_text
0,Sinh non,Sinh non là gì Phân loại mức độ sinh non Dấu h...,sinh non là gì phân loại mức độ sinh non dấu h...
1,Sinh non,Việt Nam theo các tài liệu báo cáo chung tình ...,việt nam theo các tài liệu báo cáo chung tình ...
2,Sinh non,hiện đại nhất Tình trạng của trẻ dần chuyển bi...,hiện đại nhất tình trạng của trẻ dần chuyển bi...
3,Sinh non,cơ tử vong cao vì thiếu chăm sóc cơ bản như gi...,cơ tử vong cao vì thiếu chăm sóc cơ bản như gi...
4,Sinh non,nặng nhọc thời kỳ mang thai Độ tuổi cũng tác đ...,nặng nhọc thời kỳ mang thai độ tuổi cũng tác đ...


In [37]:
df.to_csv(f"{PROCESSED_PATH}/corpus_clean.csv",index=False)